# Captain Cook: the fabulous recipes explorator

**This notebook shows how we obtain and analyzed the data for the cooking recipes dataset.**
Some scripting with `PERL` and `BASH`was needed for the cleaning part, please have a look in **DataCLeaning.iypnb** to understand the data crawling/cleaning.

In [1]:
# Basic imports
import re
import os.path
import numpy as np
import scipy as sp
import pandas as pd

# Map-related imports
import json
import branca
import folium
from pandas.io.json import json_normalize
from IPython.core.display import display, HTML

# Plot-related imports
import seaborn as sns
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets.embed import embed_minimal_html

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [2]:
# General parameters
%matplotlib inline
plt.style.use('seaborn')#switch to seaborn style
plt.rcParams["figure.figsize"] = [16,10]

DATA_FOLDER = './data/'

# 1. Data Loading
  
The Data has been fetched and cleaned with `BASH`scripts, please look in the *dataCleaning* section to understand how this was achieved.  

**Home made fetched dataset:**

In [3]:
# Importing ingredients to Pandas DF
allrecipes_df = pd.read_csv(DATA_FOLDER + 'allrecipes.csv', sep='\t',  header=None, encoding = "utf-8")
allrecipes_df.columns = ['ID', 'Region', 'Title', 'Ingredients', 'kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol']

# Bug?? need to convert into numeric somes, TODO EFFICIENT WAY TO DO THIS???
allrecipes_df['kcal'] = pd.to_numeric(allrecipes_df['kcal'], errors='coerce')
allrecipes_df['carb'] = pd.to_numeric(allrecipes_df['carb'], errors='coerce') / 1000.0 # convert to g
allrecipes_df['fat'] = pd.to_numeric(allrecipes_df['fat'], errors='coerce') / 1000.0 # convert to g
allrecipes_df['protein'] = pd.to_numeric(allrecipes_df['protein'], errors='coerce')
allrecipes_df['sodium'] = pd.to_numeric(allrecipes_df['sodium'], errors='coerce') / 1000.0
allrecipes_df['cholesterol'] = pd.to_numeric(allrecipes_df['cholesterol'], errors='coerce')

# Remove any rows which isn't properly formatted
allrecipes_df = allrecipes_df.dropna()

# Remove any duplicated lines
allrecipes_df = allrecipes_df.drop_duplicates().set_index('ID')

# Printing
allrecipes_df.head(5)

,Region,Title,Ingredients,kcal,carb,fat,protein,sodium,cholesterol
ID,,,,,,,,,
b9705d990df6857f20756fc996a54b63,us,Traditional Indiana Persimmon Pudding,2 cups persimmon pulp |2 eggs |1 cup white sug...,278.0,53.9,3.8,7.8,0.224,35.0
4658708d644d7b446d843fed5ddf60c4,us,Fish Tacos,1 cup all-purpose flour |2 tablespoons cornsta...,409.0,43.0,18.8,17.3,0.407,54.0
beed004e2a1772ba0db9da913f54122e,us,Wisconsin Slow Cooker Brats,8 bratwurst |2 (12 fluid ounce) cans or bottle...,377.0,12.8,27.4,13.8,1.046,69.0
96353c72421bd74096277c6cf8b17097,us,Buffalo Chicken Wing Sauce,2/3 cup hot pepper sauce (such as Frank&#39;s ...,104.0,0.4,11.6,0.2,0.576,31.0
ee659a6a5e69834b60744cc3e103729e,us,Minnesota's Favorite Cookie,"1 cup butter, softened |1 1/2 cups brown sugar...",140.0,14.9,8.7,1.5,0.076,22.0


In [4]:
# Importing descriptions to Pandas DF
allrecipes_desc_df = pd.read_csv(DATA_FOLDER + 'allrecipes_desc.csv', sep='£',  header=None, encoding = "utf-8",  engine='python')
allrecipes_desc_df.columns = ['ID', 'Description']

# Remove any duplicated lines
allrecipes_desc_df = allrecipes_desc_df.drop_duplicates().set_index('ID')

allrecipes_desc_df.head(5)

,Description
ID,
b9705d990df6857f20756fc996a54b63,Preheat the oven to 350 degrees F (175 degree...
4658708d644d7b446d843fed5ddf60c4,"To make beer batter: In a large bowl, combine..."
beed004e2a1772ba0db9da913f54122e,"Place bratwurst, beer, onion, and ketchup in ..."
96353c72421bd74096277c6cf8b17097,"Combine the hot sauce, butter, vinegar, Worce..."
ee659a6a5e69834b60744cc3e103729e,Preheat oven to 350 degrees F (175 degrees C)...


In [5]:
print("Number of recipes:", len(allrecipes_df.index.unique()))

Number of recipes: 15894


**Provided Dataset**

This dataset was provided with the assignment and cleaned with the provided `Perl` scripts. 

Thanks to the scripts, we obtain two datasets:

1. `cleaned_ing.csv` contains the list of ingredients for each recipe,
2. `cleaned_nutri.csv` contains the corresponding nutritional values.

Our objective is to merge these two sets to obtain a unique set with all useful informations.

In [6]:
# Importing ingredients to Pandas DF
ing_df = pd.read_csv(DATA_FOLDER + 'cleaned_ing.csv', sep='\t',  header=None, encoding = "utf-8")
ing_df.columns = ['ID', 'Title', 'Ingredients']

# Importing nutritional values to Pandas DF
nutri_df = pd.read_csv(DATA_FOLDER + 'cleaned_nutri.csv', sep='\t',  header=None, encoding = "utf-8")
nutri_df.columns = ['ID', 'kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol']

# Merging
ing_df = ing_df.set_index('ID')
nutri_df = nutri_df.set_index('ID')
provided_df = ing_df.merge(nutri_df, on='ID', how='inner')

# Drop NaNs and duplicate lines
provided_df = provided_df.dropna().drop_duplicates()

provided_df.head()

,Title,Ingredients,kcal,carb,fat,protein,sodium,cholesterol
ID,,,,,,,,
38e1b80017526d6e59ed3f986c35a43a,T.G.I. Friday's Jack Daniels Sauce Recipe #10265,1 teaspoon onion powder|1 tablespoon Tabasco s...,?,?,?,?,?,?
a3636a4dab434fe21fbcdceba7d6fcf2,Simple Peanut Squash Recipe,1 butternut squash|2 tablespoons brown sugar|1...,536,86.4,23.6,6.5,483,61
117f3c214e9de550a157ce5ee1f1cceb,Hash Brown Breakfast Casserole Recipe,"1 lb ground sausage (""hot"" or ""sage"" flavored)...",660.4,24.7,47.3,32.5,1248.0,251.6
fde8f280a690fb8bc77c10a7193db08b,Basic Homemade Country Sausage Recipe,2 pounds lean pork|1/2 pound pork fatback|3 te...,?,?,?,?,?,?
714df642f50b9ae489d285e16b59bf7b,Spinach Frittata Recipe,1 cup fresh spinach|2 egg whites|1 egg yolk|1/...,?,?,?,?,?,?


We can observe that some nutritional values are missing, which can be solved either by removing the lines or by trying to calculate these values from the given ingredients.

As trying to calculate the values from ingredients with different units (i.e. grams, cups, tbsp, etc) requires a set of informations that we do not have, we decided to leave these lines as they are for now. 

In [7]:
# Bug?? need to convert into numeric somes, TODO EFFICIENT WAY TO DO THIS???
provided_df['kcal'] = pd.to_numeric(provided_df['kcal'], errors='coerce')
provided_df['carb'] = pd.to_numeric(provided_df['carb'], errors='coerce')
provided_df['fat'] = pd.to_numeric(provided_df['fat'], errors='coerce')
provided_df['protein'] = pd.to_numeric(provided_df['protein'], errors='coerce')
provided_df['sodium'] = pd.to_numeric(provided_df['sodium'], errors='coerce')
provided_df['cholesterol'] = pd.to_numeric(provided_df['cholesterol'], errors='coerce')

# Insert Region column to match the other DF
provided_df.insert(loc=1, column='Region', value=np.nan)
provided_df.head(5)

,Title,Region,Ingredients,kcal,carb,fat,protein,sodium,cholesterol
ID,,,,,,,,,
38e1b80017526d6e59ed3f986c35a43a,T.G.I. Friday's Jack Daniels Sauce Recipe #10265,NaN,1 teaspoon onion powder|1 tablespoon Tabasco s...,NaN,NaN,NaN,NaN,NaN,NaN
a3636a4dab434fe21fbcdceba7d6fcf2,Simple Peanut Squash Recipe,NaN,1 butternut squash|2 tablespoons brown sugar|1...,536.0,86.4,23.6,6.5,483.0,61.0
117f3c214e9de550a157ce5ee1f1cceb,Hash Brown Breakfast Casserole Recipe,NaN,"1 lb ground sausage (""hot"" or ""sage"" flavored)...",660.4,24.7,47.3,32.5,1248.0,251.6
fde8f280a690fb8bc77c10a7193db08b,Basic Homemade Country Sausage Recipe,NaN,2 pounds lean pork|1/2 pound pork fatback|3 te...,NaN,NaN,NaN,NaN,NaN,NaN
714df642f50b9ae489d285e16b59bf7b,Spinach Frittata Recipe,NaN,1 cup fresh spinach|2 egg whites|1 egg yolk|1/...,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print("Number of recipes:", len(provided_df.index.unique()))

Number of recipes: 31376


In [9]:
# Concatenate the 2 DF and drop any duplicated lines, it is possible since some data come from the same website!
recipes_df = allrecipes_df.append(provided_df, sort=False).drop_duplicates()
recipes_df['Region'] = recipes_df['Region'].astype('category')

recipes_df.head()

,Region,Title,Ingredients,kcal,carb,fat,protein,sodium,cholesterol
ID,,,,,,,,,
b9705d990df6857f20756fc996a54b63,us,Traditional Indiana Persimmon Pudding,2 cups persimmon pulp |2 eggs |1 cup white sug...,278.0,53.9,3.8,7.8,0.224,35.0
4658708d644d7b446d843fed5ddf60c4,us,Fish Tacos,1 cup all-purpose flour |2 tablespoons cornsta...,409.0,43.0,18.8,17.3,0.407,54.0
beed004e2a1772ba0db9da913f54122e,us,Wisconsin Slow Cooker Brats,8 bratwurst |2 (12 fluid ounce) cans or bottle...,377.0,12.8,27.4,13.8,1.046,69.0
96353c72421bd74096277c6cf8b17097,us,Buffalo Chicken Wing Sauce,2/3 cup hot pepper sauce (such as Frank&#39;s ...,104.0,0.4,11.6,0.2,0.576,31.0
ee659a6a5e69834b60744cc3e103729e,us,Minnesota's Favorite Cookie,"1 cup butter, softened |1 1/2 cups brown sugar...",140.0,14.9,8.7,1.5,0.076,22.0


In [10]:
print("Number of total recipes:", len(recipes_df.index.unique()))

Number of total recipes: 46999


In [11]:
len(recipes_df[recipes_df['Region']=='italian'])/365

6.841095890410959

We see that the total number of recipes is enough to eat italian recipes everyday for almost 7 years!!  
We can save this DataFrame to be use later on.

In [12]:
recipes_df.to_csv(DATA_FOLDER + 'recipes_df.csv', sep='\t', encoding='utf-8')

# 2. Ingredient parsing
The cleaning is presented in `DataCleaning.ipynb`, here we use directly the result which is a list of good ingredient that can be matched in the recipes and are relevant for any statistical analysis

In [13]:
# Load csv file 'hand' cleaned
with open(DATA_FOLDER + 'cleaned_list') as f:
    ing_list = f.read().splitlines()
    
# Load the dictionnary that correct the name mispelled
ing_dict = np.load(DATA_FOLDER + 'ing_dict.npy').item()

In [14]:
# USEFUL
recipes_copy = recipes_df.copy()
recipes_copy['Ingredients'] = recipes_copy['Ingredients'].str.lower()

# Remove non alphabetic values expect of '|' which is the seperating char
recipes_copy['Ingredients'] = recipes_copy['Ingredients'].str.replace('[^a-zA-Z ]+', ' ')

In [15]:
# This step is needed to clean the dataset ingredient!
# Function that apply the cleaning dictionnary on every word in ingredients column for each recipes
def matcher(k):
    x = (i for i in ing_dict if i in k.split(' '))
    return '|'.join(map(ing_dict.get, x))

# Cleaned!!!
recipes_copy['Cleaned_Ing'] = recipes_copy['Ingredients'].map(matcher)
recipes_copy

,Region,Title,Ingredients,kcal,carb,fat,protein,sodium,cholesterol,Cleaned_Ing
ID,,,,,,,,,,
b9705d990df6857f20756fc996a54b63,us,Traditional Indiana Persimmon Pudding,cups persimmon pulp eggs cup white sugar...,278.0,53.9,3.8,7.8,0.224,35.0,butter|flour|milk|eggs|persimmon
4658708d644d7b446d843fed5ddf60c4,us,Fish Tacos,cup all purpose flour tablespoons cornstar...,409.0,43.0,18.8,17.3,0.407,54.0,salt|pepper|flour|powder|cumin|lime|corn|mayon...
beed004e2a1772ba0db9da913f54122e,us,Wisconsin Slow Cooker Brats,bratwurst fluid ounce cans or bottles b...,377.0,12.8,27.4,13.8,1.046,69.0,onion|beef
96353c72421bd74096277c6cf8b17097,us,Buffalo Chicken Wing Sauce,cup hot pepper sauce such as frank s redhot...,104.0,0.4,11.6,0.2,0.576,31.0,salt|pepper|butter|powder|vinegar|worcestershire
ee659a6a5e69834b60744cc3e103729e,us,Minnesota's Favorite Cookie,cup butter softened cups brown sugar ...,140.0,14.9,8.7,1.5,0.076,22.0,salt|butter|flour|powder|milk|eggs|vanilla|cho...
1afe62458129f741d989b89d6ae8d08c,us,Kendra's Maid-Rite Sandwiches,pounds ground beef chopped onion cup ket...,244.0,27.0,8.9,13.4,0.483,34.0,salt|onion|vinegar|beef|worcestershire|hamburger
7b93a05f2c4a9ff967957f62fe5c6622,us,Amish White Bread,cups warm water degrees f degrees c cu...,168.0,30.7,2.9,4.4,0.147,0.0,salt|flour|water|bread|yeast
fb0673097182df2dd7f7355b0e41693a,us,Buffalo Chicken Dip,ounce cans chunk chicken drained oun...,284.0,8.6,22.6,11.1,0.552,54.0,pepper|chicken|cream|celery|cheddar|cracker
1eaa22e1674b18dfbd30ec384931e9b1,us,Roast Sticky Chicken-Rotisserie Style,teaspoons salt teaspoons paprika teaspoo...,586.0,3.7,34.3,61.7,1.351,194.0,salt|pepper|onion|powder|onion|chicken


----
**The ingredient list is now perfectly cleaned: we can do some neat analysis on it**

In [18]:
# Expand the cleaned ingredient for each recipes
matrix = recipes_copy['Cleaned_Ing'].str.get_dummies('|')
matrix.head()

,achiote,acini,adobo,adzuki,agar,aioli,ajies,ajinomoto,alcohol,alfalfa,...,yoplait,yucca,zapallo,zape,zesty,zinfandel,ziploc,ziti,zucchini,zwieback
ID,,,,,,,,,,,,,,,,,,,,,
b9705d990df6857f20756fc996a54b63,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4658708d644d7b446d843fed5ddf60c4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
beed004e2a1772ba0db9da913f54122e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96353c72421bd74096277c6cf8b17097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ee659a6a5e69834b60744cc3e103729e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Convert to numpy array for mathematic manipulations
X = matrix.values

# Compute the adjacency of the ingredient Graphs
# Indeed, since each values belong to the set {0,1}
# we get a matrix which corresponds to the number of link
# between two ingredients, it is define by the number at a given line - column
# TODO explanation

adjacency = X.T @ X

NameError: name 'matrix' is not defined

In [ ]:
# Graph Plotting

# TODO: that would be nice https://www.curiousgnu.com/reddit-comments



-----
**Test with Holoviews**

In [ ]:
df = pd.DataFrame(data=adjacency,    # values
...               index=matrix.columns,    # 1st column as index
...               columns=matrix.columns)  # 1st row as the column names

df = df.unstack().to_frame().reset_index().drop_duplicates()
df.columns = ['src', 'trg', 'number']
df = df.nlargest(200, columns='number')
df

In [ ]:
#import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh')
%output fig='html' size=300

%opts Chord [label_index='index' color_index='src' edge_color_index='src'] 
%opts Chord (cmap='Category20' edge_cmap='Category20')
hv.Chord(df)

---

In [ ]:
recipes_copy.head()

47000 - recipes_copy['Region'].isnull().sum()


In [ ]:
# Count occurences
ing_ds = recipes_copy['Ingredients'].str.split(" ", expand=True) \
                                    .stack() \
                                    .map(ing_dict) \
                                    .value_counts()  \
                                    .to_frame()  \
                                    .reset_index()

# TODO THIS SHOULD BE MOVE BELOW, IT IS AN ANALYSIS -> Quite useless but hey :P
ing_ds.head()

**Try visualization of WordCloud**

In [ ]:
ing_ds_region = recipes_copy

regions = ing_ds_region['Region'].unique().dropna()

In [ ]:
top_5_regions = ing_ds_region.groupby('Region').count().sort_values(by='Title', ascending=False).head(5)\
.index.tolist()

top_10_regions = ing_ds_region.groupby('Region').count().sort_values(by='Title', ascending=False).head(10)\
.index.tolist()

top_15_regions = ing_ds_region.groupby('Region').count().sort_values(by='Title', ascending=False).head(15)\
.index.tolist()

In [ ]:
recipes_regions = ing_ds_region.groupby('Region').count().sort_values(by='Title', ascending=False)['Title']

recipes_regions = recipes_regions.to_frame()

recipes_regions.columns = ['Recipe Count']

recipes_regions.plot.bar()

plt.savefig("./website/freelancer-theme/img/recipe_count_per_region.svg", format='svg')

plt.show()

In [ ]:
df = recipes_copy

titles = df.Title.tolist()

titles = ''.join(titles)

wordcloud = WordCloud(background_color="white", mode="RGBA", max_words=1000, width=800, height=400)\
.generate(titles)

# create coloring from image
plt.figure(figsize=[10,10])
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")

# store to file
plt.savefig("./website/freelancer-theme/img/Flags/titles_wordcloud.svg", format="svg", dpi=700)

plt.show()

In [ ]:
ingredients_per_region_list = []
for region in regions:
    ing_ds_region_2 = ing_ds_region[ing_ds_region['Region'] == region]['Ingredients'].str.split(" ", expand=True) \
                                                                    .stack() \
                                                                    .map(ing_dict) \
                                                                    .value_counts()  \
                                                                    .to_frame()  \
                                                                    .reset_index()
    ing_ds_region_2['Region'] = region
    ingredients_per_region_list.append(ing_ds_region_2)

ingredients_per_region = pd.concat(ingredients_per_region_list)
ingredients_per_region = ingredients_per_region.set_index('Region')
ingredients_per_region.columns = ['Ingredient', 'Count']

In [ ]:
def get_region_ing(region):
    ingredients = ingredients_per_region[ingredients_per_region.index == region]
    return dict(zip(ingredients['Ingredient'], ingredients['Count']))
    

for region in top_5_regions:
    ing_list = get_region_ing(region)
    # Generate a word cloud image
    mask = np.array(Image.open("Masks/"+region+".png"))

    wordcloud = WordCloud(background_color="white", mode="RGBA", max_words=1000, mask=mask, width=800, height=400)\
    .generate_from_frequencies(ing_list)

    print(mask.shape)
    # create coloring from image
    image_colors = ImageColorGenerator(mask)
    plt.figure(figsize=[10,10])
    plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear")
    plt.axis("off")

    # store to file
    plt.savefig("./website/freelancer-theme/img/Flags/"+region+"_ing.svg", format="svg")

    plt.show()


**Try visualization with holoviews**

In [ ]:
df = ingredients_per_region.reset_index()
top_ingredients_list = []

for region in regions:
    # Select 50 most used ingredients
    top_ingredients = df[df['Region'] == region].head(50)
    top_ingredients_list.append(top_ingredients)
    
top_ingredients = pd.concat(top_ingredients_list)


def compare_lists_ing(list1, list2):
    '''This function finds how many ingredients are in both lists'''
    counter = 0
    for ingredient in list1:
        if ingredient in list2:
            counter += 1
    return counter

coeff = []
region_1 = []
region_2 = []

# Create a list of coefficients between different regions
for i, region1 in enumerate(top_15_regions):
    for j, region2 in enumerate(top_15_regions):
        if region1 != region2:
            list1 = top_ingredients[top_ingredients['Region'] == region1]['Ingredient'].tolist()
            list2 = top_ingredients[top_ingredients['Region'] == region2]['Ingredient'].tolist()
            
            region_1.append(region1)
            region_2.append(region2)
            coeff.append(compare_lists_ing(list1, list2))
            

In [ ]:
from math import log
coeff_log = [int(0.001*(x)**4+0.001*(x)**2-400) for x in coeff]

df = pd.DataFrame({'Region 1': region_1, 'Region 2': region_2, 'Coeff': coeff_log})  

top_regions_df = pd.DataFrame({'Region': top_15_regions})

top_regions_df, df

In [ ]:
import holoviews as hv
hv.extension('bokeh')
%output fig='html' size=200

%opts Chord [label_index='Region' color_index='Region' edge_color_index='Region 2'] 
%opts Chord (cmap='Category20' edge_cmap='Category20')
nodes = hv.Dataset(top_regions_df, 'Region')

chord = hv.Chord((df, nodes), ['Region 1', 'Region 2'], ['Coeff'])

renderer = hv.renderer('bokeh')

plot = renderer.get_plot(chord).state

from bokeh.io import output_file, save, show
save(plot, './website/freelancer-theme/img/ingredients_chord.html')
show(plot)

# 3. Cooking time study-case

In this part we would like to analyze the cooking time of the recipes to be able to classify which regions have the highest and lowest cooking time.

In [ ]:
# Extract all timing from recipes
timing_df = allrecipes_desc_df['Description'].str.extractall(r'(\d+) minutes|(\d+) hour|hours')
timing_df.columns = ['minutes', 'hours']

#Replace Nan by 0 and switch to int type
timing_df = timing_df.fillna(0).astype(int)

#Sum the number of minutes to get the recipe time
timing_df['Time (min)'] = timing_df['minutes']+timing_df['hours']*60

timing_df.head()

In [ ]:
# Sum the total amount of time for each recipe
time_recipe = timing_df.groupby('ID').agg('sum')
time_recipe = time_recipe.drop(['minutes','hours'], axis=1)

time_recipe.head()

# 4. Merging
Finally, we can merge everything to a single DataFrame to use it for Visualization

In [ ]:
# Merging Cooking Time
cleaned_df = recipes_df.merge(time_recipe, on='ID', how='left')

# Cleaning ingredient and ingredient substition
# This is not yet implemented but we are close to achieve this

cleaned_df.sample(5)

# 5. Analysis

This part presents some basic statistical analysis of the data.

First we analyse the data by region and observe *mean*, *median*, *min* and *max* for each nutritional value.

In [ ]:
# Some classic analysis
stats_regions = cleaned_df.groupby('Region')
stats_regions = stats_regions.agg({'kcal' : ['mean', 'median', 'min', 'max'],
                                       'carb' : ['mean', 'median', 'min', 'max'],
                                       'fat' : ['mean', 'median', 'min', 'max'],
                                       'protein' : ['mean', 'median', 'min', 'max'],
                                       'sodium' : ['mean', 'median', 'min', 'max'],
                                       'cholesterol' : ['mean', 'median', 'min', 'max'],
                                       'Time (min)' : ['mean', 'median', 'min', 'max']})
stats_regions.sort_values([('kcal', 'mean')], ascending=False).head()

# 6. Visualization

In this part we present the overall visualization of informations we retrieve in the dataset.

###  Plots

In [ ]:
# Interactive plot of correlation between nutritive values 
def f(nutritive1, nutritive2):
    
    sns.set_context("notebook", font_scale=1.5)
    sns.scatterplot(cleaned_df[nutritive1], cleaned_df[nutritive2])
    plt.show()
    
# Interact
interact(f, nutritive1=['kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol'],
            nutritive2=['kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol']);

In the plot above we can see the correlation between the different nutritional values. 

For example, there are many recipes where high carbs and fats correspond to high caloric plates, but less so for high proteins. Also it would seem that fats and cholesterol are not as correlated as we would think.

Below is a plot that shows the correlation coefficient for pairs of nutritional values by region. 

In [ ]:
# Correlation between nutritional values shown per region
def f(region):
    sns.set_context("notebook", font_scale=1.5)
    
    # .iloc[:,:-1] is to avoid the Time column
    # It can be interesting to see if there is a correlation
    corr = cleaned_df.iloc[:,:-1][cleaned_df['Region'] == region].corr()
    sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()
    
# Interact
interact(f, region=cleaned_df.Region.unique().dropna());

Below is a plot that shows the statistics by nutritional or time value of recipes classified by region. The plot is automatically ordered by median, so we obtain the region with the highest median for that item value.

In [ ]:
# Item value statistics by regions
def f(item):
    recipe_sorted = stats_regions.sort_values([(item, 'median')], ascending=False)

    sns.set_context("notebook", font_scale=1.5)
    sns.boxplot(cleaned_df[item], cleaned_df['Region'], order=recipe_sorted.index)
    
    ## There is a big outlier for Sodium & Time, we will handle it later
    if(item == 'sodium'):
        plt.xlim(-0.5, 10)
        
    if(item == 'Time (min)'):
        plt.xlim(-50, 1500)
    ##
    plt.show()
    
# Interact
plt_interact = interact(f, item=['kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol', 'Time (min)']);
embed_minimal_html('export.html', views=[plt_interact], title='Widgets export')

We can see that the most calorical, fat and protein rich recipes belong to Malaysia, while the sodium intake is won by the korean recipes. The ones that have to be most careful about the cholesterol intake seem to be the French.

By comparing the median we also see that the longest cooking time and preparation in total is for Persian recipes, whereas Japanese's recipes are the shortest

### Maps

In [ ]:
# Loading JSON of world map
map_recipes_json = json.load(open(DATA_FOLDER + 'recipes_map.json'))

In [ ]:
def layer_colormap(topojson, df, column, colorscale):
    
    # Create a layer
    feature_map = folium.FeatureGroup(name=column, overlay=False)  
    
    def style_function(feature):
    # Fetching values for the mean of the category for the given asked continent
        value = df[df['Region'] == feature['properties']['Region']][column].mean()
        return {
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.5,
            'fillColor': '#black' if np.isnan(value) else colorscale(value)
                }
    # Fetch values from the DataFrame and apply the colormap to the values
    # If the value is NaN, the corresponding color is dark-grey
    folium.GeoJson(topojson, style_function=style_function).add_to(feature_map)

    return feature_map;

In [ ]:
# Create a new empty map
map_info  = folium.Map([30,0], tiles='cartodbpositron', zoom_start=2)

# Add for each nutritive information the map
for category in ['kcal','carb','fat','protein','sodium','cholesterol', 'Time (min)']:
    colorscale = branca.colormap.linear.YlOrRd_09.scale((min(stats_regions[category]['mean'])), max(stats_regions[category]['mean']))
    layer_colormap(map_recipes_json, cleaned_df, category, colorscale).add_to(map_info)
    
# Add a legend to the colormap and append it to the base layer
colorscale.caption = 'Mean of the nutritive value selected'
map_info.add_child(colorscale) 

# Adding the tile Layer thus it is prettier
folium.TileLayer(tiles='cartodbpositron', overlay=True).add_to(map_info)

# Layer Control to select the different layer created before
folium.LayerControl(collapsed=False, position='bottomleft').add_to(map_info);

# Save/Display
map_info.save('map_info.html')
#map_info

In [ ]:
%%HTML
<iframe src="map_info.html" width=100% height=700></iframe>

On the previous map, we can see how the different nutritive properties of the recipes vary through the different continents. We can thus see some correlations like the kcal of the recipe and the fat which are both high in the same continents.  

**Note:** we actually have a small issue with the colormap and we will be fixing it by using a different kind of interactive map to show more interesting information (Ingredients distribution, min/max or median for nutrition)